In [1]:
pip install keras==2.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.2/170.2 KB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.1/578.1 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 KB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.1/578.1 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.8/511.8 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 KB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.8/511.8 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.0/498.0 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━

In [2]:
pip install mia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for mia: filename=mia-0.1.2-py3-none-any.whl size=11106 sha256=ebfdb65b6ac0488af380408dd1d05ef444ba441d6eeeac54594c98b9868eaeec
  Stored in directory: /root/.cache/pip/wheels/c7/fb/77/a632189442690e610b8fad3eea1887996438e0ffc3cdadc039
Successfully built mia


In [66]:
import numpy as np
import tensorflow as tf
import random
import datetime
#import tensorflow_privacy
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import keras

In [67]:
def load_mnist():
  """Loads MNIST and preprocesses to combine training and validation data."""
  train, test = tf.keras.datasets.mnist.load_data()
  #train, test = tf.keras.datasets.fashion_mnist.load_data()
  #train, test = tf.keras.datasets.cifar10.load_data()
  train_data, train_labels = train
  test_data, test_labels = test

  train_data = np.array(train_data, dtype=np.float32) / 255
  test_data = np.array(test_data, dtype=np.float32) / 255

  train_data = train_data.reshape((train_data.shape[0], 28, 28, 1))
  test_data = test_data.reshape((test_data.shape[0], 28, 28, 1))
  #train_data = train_data.reshape((train_data.shape[0], 32, 32, 3))
  #test_data = test_data.reshape((test_data.shape[0], 32, 32, 3))

  train_labels = np.array(train_labels, dtype=np.int32)
  test_labels = np.array(test_labels, dtype=np.int32)

  #train_labels = np.array(train_labels, dtype=np.int64)
  #test_labels = np.array(test_labels, dtype=np.int64)

  train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
  test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

  #assert train_data.min() == 0.
  #assert train_data.max() == 1.
  #assert test_data.min() == 0.
  #assert test_data.max() == 1.

  return train_data, train_labels, test_data, test_labels

In [68]:
train_data, train_labels, test_data, test_labels = load_mnist()

In [69]:
cifar_train = train_data, train_labels
cifar_test = test_data, test_labels

In [70]:
CLIENTS = 3
SIZE = 10000

def get_data(source):   
    
    all_data = (np.array(source[0][:SIZE*CLIENTS]), source[1][:SIZE*CLIENTS]) 
    
    split_data = []
    for s in range(CLIENTS):
        start = s*SIZE
        end = s*SIZE + SIZE
        split_data.append((all_data[0][start:end], all_data[1][start:end]))
    
    external_data = (np.array(source[0][SIZE*CLIENTS:]), source[1][SIZE*CLIENTS:]) 
    
    return all_data, split_data, external_data

In [71]:
CLIENTS = 3
SIZE = 1000

def get_test_data(source):   
    
    all_data = (np.array(source[0][:SIZE*CLIENTS]), source[1][:SIZE*CLIENTS]) 
    
    split_data = []
    for s in range(CLIENTS):
        start = s*SIZE
        end = s*SIZE + SIZE
        split_data.append((all_data[0][start:end], all_data[1][start:end]))
    
    external_data = (np.array(source[0][SIZE*CLIENTS:]), source[1][SIZE*CLIENTS:]) 
    
    return all_data, split_data, external_data

In [72]:
cifar_train_data, cifar_train_fed_data, attacker_data = get_data(cifar_train)

In [73]:
cifar_test_data, cifar_test_fed_data, externat_test_data = get_test_data(cifar_test)

In [76]:
def create_compiled_keras_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=[28, 28, 1]),
        tf.keras.layers.Dense(300, activation="relu"),
        tf.keras.layers.Dense(100, activation="relu"),
        tf.keras.layers.Dense(10, activation="softmax")
    ])
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
    model.compile(loss=loss,
              optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01),
              metrics=["accuracy"])
    return model

In [77]:
teacher_model = create_compiled_keras_model()

In [78]:
student_model = create_compiled_keras_model()

In [79]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)

            # Compute scaled distillation loss from https://arxiv.org/abs/1503.02531
            # The magnitudes of the gradients produced by the soft targets scale
            # as 1/T^2, multiply them by T^2 when using both hard and soft targets.
            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

In [80]:
initial_model = create_compiled_keras_model()

FedTrain = []
FedTest = []
FedAcc = []

DistTrain = []
DistTest = []
DistAcc = []

Dstudent_loss = []
Ddistillation_loss = []
Daccuracy = []

for r in range(18):
    
    deltas = []

    for c in range(CLIENTS):

        teacher_model = create_compiled_keras_model()

        teacher_model.set_weights(initial_model.get_weights())

        #fed_history_callback = federated_model.fit(cifar_train_fed_data[c][0], cifar_train_fed_data[c][1], 
         #                                      batch_size=250, epochs=10, verbose=1)
        fed_history_callback = teacher_model.fit(cifar_train_fed_data[c][0], cifar_train_fed_data[c][1], 
                                               batch_size=32, epochs=10, verbose=1)
        
        delta = np.array(initial_model.get_weights()) - np.array(teacher_model.get_weights())

        deltas.append(delta)

    print('Epoch {}/18'.format(r+1))
    delt_av = (deltas[0] + deltas[1] + deltas[2]) / 3
    new_weights = np.array(initial_model.get_weights()) - delt_av
    initial_model.set_weights(new_weights)
    
    FedTrain.append(initial_model.evaluate(cifar_train_data[0], cifar_train_data[1])[0])
    validation = initial_model.evaluate(cifar_test_data[0], cifar_test_data[1])
    FedTest.append(validation[0])
    FedAcc.append(validation[1])

    distiller = Distiller(student=student_model, teacher=teacher_model)
    distiller.compile(
    optimizer=keras.optimizers.SGD(),
    metrics=["accuracy"],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
    )
    # Distill teacher to student
    dist = distiller.fit(cifar_train_data[0], cifar_train_data[1], epochs=10)
    Dstudent_loss.append(dist.history['student_loss'])
    Ddistillation_loss.append(dist.history['distillation_loss'])
    Daccuracy.append(dist.history['accuracy'])

    # Evaluate student on test dataset
    #distiller.evaluate(cifar_test_data[0], cifar_test_data[1])

    DistTrain.append(distiller.evaluate(cifar_train_data[0], cifar_train_data[1])[0])
    validation = distiller.evaluate(cifar_test_data[0], cifar_test_data[1])
    DistTest.append(validation[0])
    DistAcc.append(validation[1])


Epoch 1/10
32/32 [==============================] - 1s 6ms/step - loss: 1.9280 - accuracy: 0.5970
Epoch 2/10
32/32 [==============================] - 0s 6ms/step - loss: 0.5169 - accuracy: 0.8450
Epoch 3/10
32/32 [==============================] - 0s 6ms/step - loss: 0.3320 - accuracy: 0.8840
Epoch 4/10
32/32 [==============================] - 0s 6ms/step - loss: 0.2185 - accuracy: 0.9320
Epoch 5/10
32/32 [==============================] - 0s 6ms/step - loss: 0.2766 - accuracy: 0.9390
Epoch 6/10
32/32 [==============================] - 0s 6ms/step - loss: 0.1302 - accuracy: 0.9600
Epoch 7/10
32/32 [==============================] - 0s 6ms/step - loss: 0.0647 - accuracy: 0.9830
Epoch 8/10
32/32 [==============================] - 0s 6ms/step - loss: 0.1293 - accuracy: 0.9650
Epoch 9/10
32/32 [==============================] - 0s 6ms/step - loss: 0.0774 - accuracy: 0.9760
Epoch 10/10
32/32 [==============================] - 0s 6ms/step - loss: 0.0374 - accuracy: 0.9900
Epoch 1/10


<ipython-input-80-7054eab812ff>:30: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  delta = np.array(initial_model.get_weights()) - np.array(teacher_model.get_weights())


32/32 [==============================] - 1s 5ms/step - loss: 2.3516 - accuracy: 0.5520
Epoch 2/10
32/32 [==============================] - 0s 6ms/step - loss: 0.6304 - accuracy: 0.8150
Epoch 3/10
32/32 [==============================] - 0s 5ms/step - loss: 0.4381 - accuracy: 0.8740
Epoch 4/10
32/32 [==============================] - 0s 6ms/step - loss: 0.2973 - accuracy: 0.9160
Epoch 5/10
32/32 [==============================] - 0s 6ms/step - loss: 0.2105 - accuracy: 0.9400
Epoch 6/10
32/32 [==============================] - 0s 5ms/step - loss: 0.2012 - accuracy: 0.9570
Epoch 7/10
32/32 [==============================] - 0s 6ms/step - loss: 0.1164 - accuracy: 0.9700
Epoch 8/10
32/32 [==============================] - 0s 6ms/step - loss: 0.1873 - accuracy: 0.9630
Epoch 9/10
32/32 [==============================] - 0s 5ms/step - loss: 0.1713 - accuracy: 0.9590
Epoch 10/10
32/32 [==============================] - 0s 6ms/step - loss: 0.0858 - accuracy: 0.9770
Epoch 1/10
32/32 [============

<ipython-input-80-7054eab812ff>:36: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  new_weights = np.array(initial_model.get_weights()) - delt_av


94/94 [==============================] - 0s 3ms/step - loss: 0.6788 - accuracy: 0.8180
Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/backend.py:4869: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


94/94 [==============================] - 1s 8ms/step - accuracy: 0.1550 - student_loss: 2.2818 - distillation_loss: 0.0457
Epoch 2/10
94/94 [==============================] - 1s 8ms/step - accuracy: 0.2307 - student_loss: 2.2009 - distillation_loss: 0.0448
Epoch 3/10
94/94 [==============================] - 1s 8ms/step - accuracy: 0.3293 - student_loss: 2.1266 - distillation_loss: 0.0439
Epoch 4/10
94/94 [==============================] - 1s 8ms/step - accuracy: 0.4057 - student_loss: 2.0492 - distillation_loss: 0.0428
Epoch 5/10
94/94 [==============================] - 1s 8ms/step - accuracy: 0.4753 - student_loss: 1.9724 - distillation_loss: 0.0417
Epoch 6/10
94/94 [==============================] - 1s 8ms/step - accuracy: 0.5370 - student_loss: 1.8966 - distillation_loss: 0.0404
Epoch 7/10
94/94 [==============================] - 1s 8ms/step - accuracy: 0.5757 - student_loss: 1.8177 - distillation_loss: 0.0391
Epoch 8/10
94/94 [==============================] - 1s 8ms/step - accurac

In [81]:
np.mean(dist.history['student_loss'])

0.15730745494365692

In [82]:
np.mean(dist.history['distillation_loss'])

0.003269837237894535

In [83]:
np.mean(dist.history['accuracy'])

0.9454666614532471

In [84]:
np.mean(DistTrain)

0.8984259366989136

In [22]:
np.mean(DistTest)

0.8268518480989668

In [16]:
np.mean(DistAcc)

0.6823559204737345

In [23]:
np.mean(Dstudent_loss)

0.5154598756382863

In [24]:
np.mean(Ddistillation_loss)

0.003998374488340535

In [85]:
np.mean(Daccuracy)

0.8792629619439443

**Membership Inference Attack**

In [86]:
import numpy as np

from absl import app
from absl import flags

import tensorflow as tf
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from mia.estimators import ShadowModelBundle, AttackModelBundle, prepare_attack_data

In [87]:
NUM_CLASSES = 10

SHADOW_DATASET_SIZE = 1000
ATTACK_TEST_DATASET_SIZE = 5000

num_shadows = 10
#num_shadows = 1


In [88]:
def target_model_fn():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=[28, 28, 1]),
        tf.keras.layers.Dense(300, activation="relu"),
        tf.keras.layers.Dense(100, activation="relu"),
        tf.keras.layers.Dense(10, activation="softmax")
    ])
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
    model.compile(loss=loss,
              optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.01),
              metrics=["accuracy"])
    return model

In [89]:
def attack_model_fn():
    model = tf.keras.models.Sequential()

    model.add(layers.Dense(64, activation="relu", input_shape=(10,)))
    model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
    model.add(layers.Dense(32, activation="relu"))
    model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
    model.add(layers.Dense(32, activation="relu"))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile("adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [90]:
target_model = student_model

In [91]:
# Train the shadow models.
smb = ShadowModelBundle(
    target_model_fn,
    shadow_dataset_size=SHADOW_DATASET_SIZE,
    num_models=num_shadows
)

# Using cifar10 test set to train shadow models
attacker_X_train, attacker_X_test, attacker_y_train, attacker_y_test = train_test_split(
    cifar_test[0], cifar_test[1], test_size=0.5)

print(attacker_X_train.shape, attacker_X_test.shape)

print("Training the shadow models...")
X_shadow, y_shadow = smb.fit_transform(
    attacker_X_train,
    attacker_y_train,
    fit_kwargs=dict(
        epochs=32,
        verbose=True,
        validation_data=(attacker_X_test, attacker_y_test)
    )
)

(5000, 28, 28, 1) (5000, 28, 28, 1)
Training the shadow models...
Epoch 1/32
32/32 [==============================] - 1s 20ms/step - loss: 1.9971 - accuracy: 0.4290 - val_loss: 1.6095 - val_accuracy: 0.6096
Epoch 2/32
32/32 [==============================] - 0s 14ms/step - loss: 1.2636 - accuracy: 0.7470 - val_loss: 1.0540 - val_accuracy: 0.7392
Epoch 3/32
32/32 [==============================] - 0s 14ms/step - loss: 0.8543 - accuracy: 0.8010 - val_loss: 0.7983 - val_accuracy: 0.7848
Epoch 4/32
32/32 [==============================] - 1s 26ms/step - loss: 0.6583 - accuracy: 0.8500 - val_loss: 0.6866 - val_accuracy: 0.8134
Epoch 5/32
32/32 [==============================] - 1s 29ms/step - loss: 0.5583 - accuracy: 0.8720 - val_loss: 0.5719 - val_accuracy: 0.8462
Epoch 6/32
32/32 [==============================] - 1s 23ms/step - loss: 0.4833 - accuracy: 0.8840 - val_loss: 0.5261 - val_accuracy: 0.8616
Epoch 7/32
32/32 [==============================] - 1s 23ms/step - loss: 0.4314 - accura

/usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/engine/sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


In [92]:
# ShadowModelBundle returns data in the format suitable for the AttackModelBundle.
amb = AttackModelBundle(attack_model_fn, num_classes=NUM_CLASSES)

# Fit the attack models.
print("Training the attack models...")
amb.fit(X_shadow, y_shadow, fit_kwargs=dict(epochs=32, verbose=True)
)

Training the attack models...
Epoch 1/32
62/62 [==============================] - 1s 5ms/step - loss: 0.6909 - accuracy: 0.5288
Epoch 2/32
62/62 [==============================] - 0s 4ms/step - loss: 0.6870 - accuracy: 0.5482
Epoch 3/32
62/62 [==============================] - 0s 4ms/step - loss: 0.6796 - accuracy: 0.5273
Epoch 4/32
62/62 [==============================] - 0s 5ms/step - loss: 0.6716 - accuracy: 0.5365
Epoch 5/32
62/62 [==============================] - 0s 4ms/step - loss: 0.6679 - accuracy: 0.5656
Epoch 6/32
62/62 [==============================] - 0s 4ms/step - loss: 0.6681 - accuracy: 0.5462
Epoch 7/32
62/62 [==============================] - 0s 4ms/step - loss: 0.6658 - accuracy: 0.5482
Epoch 8/32
62/62 [==============================] - 0s 4ms/step - loss: 0.6631 - accuracy: 0.5539
Epoch 9/32
62/62 [==============================] - 0s 4ms/step - loss: 0.6624 - accuracy: 0.5564
Epoch 10/32
62/62 [==============================] - 0s 4ms/step - loss: 0.6639 - accura

In [94]:
target_data = cifar_train_fed_data[0]
attacker_data = cifar_test_fed_data[0]

In [98]:
#attacker_data = cifar_train_data

In [99]:
attack_test_data, real_membership_labels = prepare_attack_data(target_model, target_data, attacker_data)

In [100]:
def results(attack_guesses,real_membership_labels):
    pred_labels=attack_guesses
    true_labels=real_membership_labels

    # True Positive (TP): we predict a label of 1 (positive), and the true label is 1.
    TP = np.sum(np.logical_and(pred_labels == 1, true_labels == 1))

    # True Negative (TN): we predict a label of 0 (negative), and the true label is 0.
    TN = np.sum(np.logical_and(pred_labels == 0, true_labels == 0))

    # False Positive (FP): we predict a label of 1 (positive), but the true label is 0.
    FP = np.sum(np.logical_and(pred_labels == 1, true_labels == 0))

    # False Negative (FN): we predict a label of 0 (negative), but the true label is 1.
    FN = np.sum(np.logical_and(pred_labels == 0, true_labels == 1))

    print ('TP: %i, FP: %i, TN: %i, FN: %i' % (TP,FP,TN,FN))

    print("acc = " + str((TP+TN)/(TP+TN+FP+FN)))
    print("precision = " + str((TP)/(TP+FP)))
    print("recall = " + str((TP)/(TP+FN)))
    
    acc= (TP+TN)/(TP+TN+FP+FN)
    prec=(TP)/(TP+FP)
    rec=(TP)/(TP+FN)
    return [acc,prec,rec]

In [101]:
attack_guesses = amb.predict(attack_test_data)
attack_precision = np.mean((attack_guesses == 1) == (real_membership_labels == 1))

class_precision = []

for c in range(NUM_CLASSES):
    #attack_test_data, real_membership_labels = prepare_attack_data(centralized_model, cifar_train_data, attacker_data)
    target_indices = [i for i, d in enumerate(target_data[1].argmax(axis=1)) if d == c]
    test_indices = [i for i, d in enumerate(attacker_data[1].argmax(axis=1)) if d == c]


    print(np.sum(attack_guesses[target_indices]==1) / (np.sum(attack_guesses[target_indices]) + np.sum(attack_guesses[SIZE:][test_indices])))
    
    class_precision.append(
            np.sum(attack_guesses[target_indices]==1) / (np.sum(attack_guesses[target_indices])
                                                     + np.sum(attack_guesses[SIZE:][test_indices])))
print("Average Accuracy: ", attack_precision)

result=results(attack_guesses,real_membership_labels)
 
    #attack_accuracy_class[c].append(result)

0.26479750778816197
0.27323943661971833
0.2434017595307918
0.24085365853658536
0.25274725274725274
0.250814332247557
0.22
0.25745257452574527
0.2465277777777778
0.25396825396825395
Average Accuracy:  0.3335
TP: 836, FP: 2502, TN: 498, FN: 164
acc = 0.3335
precision = 0.2504493708807669
recall = 0.836


**MIA via Prediction Sensitivity**

In [102]:
import pickle
import argparse

import numpy as np
import torch
from numpy import linalg as LA
from sklearn.metrics import precision_score, recall_score
from sklearn.cluster import SpectralClustering

In [103]:
np.random.seed(seed=14)
torch.manual_seed(14)

In [104]:
target_model = student_model
#target_model = single_model0

In [105]:
parser = argparse.ArgumentParser()
parser.add_argument('--n_sample', type=int, default=5000)
parser.add_argument('--n_attack', type=int, default=50)
parser.add_argument('--seed', type=int, default=140)
parser.add_argument('--neighbors', type=int, default=40)
parser.add_argument('--data_generate', type=bool, default=False)
attack_args = parser.parse_args(args=[])

In [106]:
precisions = []
recalls = []
f1_scores = []

In [107]:
np.random.seed(seed=attack_args.seed)
torch.manual_seed(attack_args.seed)

In [108]:
import numpy as np
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.compose import ColumnTransformer
from collections import Counter
from sklearn.model_selection import train_test_split

In [36]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/Proposal work'

Mounted at /content/drive
/content/drive/My Drive/Proposal work


In [109]:
def data_reader(data_name = "mnist"):
    file_path = "data/"
    #data = pd.read_csv(file_path + 'mnist_train.csv', header=1)
    data = pd.read_csv(file_path + 'mnist_train.csv', header=1, skiprows=30000, nrows=29999)
    #data = pd.read_csv(file_path + 'fashion-mnist_train.csv', header=1)
    #data = pd.read_csv(file_path + 'fashion-mnist_train.csv', header=1, skiprows=30000, nrows=29999)
    #data = pd.read_csv(file_path + 'fashion-mnist_test.csv', header=0)
    #data = pd.read_csv(file_path + 'cifar_train.csv', header=1, skiprows=30000, nrows=29999)
    data = np.array(data)
    labels = data[:,0]
    data = data[:,1:]
        
    categorical_features = []
    
    data = data/data.max()
    oh_encoder = ColumnTransformer(
    [('oh_enc', OneHotEncoder(sparse=False), categorical_features),], 
    remainder='passthrough' )
    oh_data = oh_encoder.fit_transform(data)
        
    #randomly select 10000 records as training data
    train_idx = np.random.choice(len(labels), 9999, replace = False)
    idx = range(len(labels))
    idx = np.array(idx)
    test_idx = list(set(idx).difference(set(train_idx)))
    test_idx = np.array(test_idx)
    
    assert test_idx.sum() + train_idx.sum() == idx.sum()
    
    X_train = data[train_idx,:]
    Y_train = labels[train_idx]
    
    X_test = data[test_idx,:]
    Y_test = labels[test_idx]
    
    orig_dataset = {"X_train":X_train,
               "Y_train":Y_train,
               "X_test":X_test,
               "Y_test":Y_test}
    
    X_train = oh_data[train_idx,:]
    
    X_test = oh_data[test_idx,:]
    
    oh_dataset = {"X_train":X_train,
               "Y_train":Y_train,
               "X_test":X_test,
               "Y_test":Y_test}

    return orig_dataset, oh_dataset, oh_encoder

In [110]:
orig_dataset, oh_dataset, OH_Encoder = data_reader("mnist")

In [111]:
class_label_for_count = np.unique(np.hstack([orig_dataset["Y_train"], orig_dataset["Y_test"]]))
n_class = len(class_label_for_count)
n_features = orig_dataset['X_train'].shape[1]

In [112]:
y_attack = np.hstack(([np.ones(int(attack_args.n_attack/2)), np.zeros(int(attack_args.n_attack/2))]))
x_attack = np.zeros((int(attack_args.n_attack), n_features))

In [113]:
Jacobian_matrix = np.zeros([attack_args.n_attack, n_class, n_features])

In [114]:
if attack_args.data_generate:
    output_x = np.zeros((attack_args.n_attack, n_features))
    output_y = y_attack
    classes = np.zeros((attack_args.n_attack, 1))

In [115]:
def fn_R_given_Selected(dataset, IN_or_OUT = 1):
    if(IN_or_OUT == 1):#IN_or_OUT == 1 meaning selecting R_given from training set
        idx = np.random.choice( len(dataset['Y_train']) )
        R_given = dataset['X_train'][idx,:]
        R_given_y = dataset['Y_train'][idx]
    elif(IN_or_OUT == 0):#IN_or_OUT == 0 meaning selecting R_given from testing set
        idx = np.random.choice( len(dataset['Y_test']) )
        R_given = dataset['X_test'][idx,:]
        R_given_y = dataset['Y_test'][idx]
    return R_given, R_given_y

In [116]:
def Target_Model_pred_fn(Target_Model, X_test):
    pred_proba = Target_Model.predict_proba(X_test)
    return pred_proba

In [117]:
categorical_list ={
    "mnist": [1,2,3,4,5,6,7,8,9,10],
}

In [118]:
def fn_Sample_Generator(R_given, dataset):
    if not dataset in categorical_list.keys():
        dataset = "null"
    epsilon = 1e-6
    R_given = R_given.reshape([1, -1])
    n_feature = R_given.shape[1]
    local_samples = np.repeat(R_given, repeats=n_feature, axis=0)
    for i in range(n_feature):
        if i in categorical_list[dataset]:
            continue
        local_samples[i][i] += epsilon

    return local_samples

In [119]:
def fn_Jacobian_Calculation(R_given, local_proba, n_features, n_class):
    epsilon = 1e-6
    jacobian = np.zeros([n_class, n_features])

    for ii in range(n_class):
        jacobian[ii, :] = (local_proba[:, ii] - R_given[ii]) / epsilon
    return jacobian

In [120]:
for ii in range(attack_args.n_attack):
      R_x, R_y = fn_R_given_Selected(orig_dataset, IN_or_OUT=y_attack[ii])
      R_x_OH = OH_Encoder.transform(R_x.reshape(1, -1))
      x_attack[ii] = R_x
      local_samples = fn_Sample_Generator(R_x, "mnist")
      oh_local_samples = OH_Encoder.transform(local_samples)
      local_proba = Target_Model_pred_fn(target_model, oh_local_samples)
      R_local_proba = Target_Model_pred_fn(target_model, R_x_OH)
      Jacobian_matrix[ii] = fn_Jacobian_Calculation(R_local_proba[0], local_proba, n_features, n_class)

      if attack_args.data_generate:
          output_x[ii] = R_x
          classes[ii] = R_y

In [122]:
Jacobian_norms = LA.norm(Jacobian_matrix, axis=(1, 2))

In [123]:
split = 1
attack_cluster = SpectralClustering(n_clusters=6, n_jobs=-1, affinity='nearest_neighbors', n_neighbors=19)
y_attack_pred = attack_cluster.fit_predict(Jacobian_norms.reshape(-1, 1))
cluster_1 = np.where(y_attack_pred >= split)[0]
cluster_0 = np.where(y_attack_pred < split)[0]
y_attack_pred[cluster_1] = 1
y_attack_pred[cluster_0] = 0
cluster_1_mean_norm = Jacobian_norms[cluster_1].mean()
cluster_0_mean_norm = Jacobian_norms[cluster_0].mean()
if cluster_1_mean_norm > cluster_0_mean_norm:
  y_attack_pred = np.abs(y_attack_pred-1)

In [124]:
precision = precision_score(y_attack, y_attack_pred)
recall = recall_score(y_attack, y_attack_pred)
f1_score = 2*precision*recall/(precision+recall)
print(precision, recall, f1_score)
precisions.append(precision)
recalls.append(recall)
f1_scores.append(f1_score)

0.42105263157894735 0.32 0.3636363636363636
